# Это шаблон ноута для теста методов кластеризации

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
# YOUR DATA DIRECTORY HERE
data_dir = '/media/arrteom/3923b309-fef1-47f0-a74c-cd259de5b45b/PAMAP2_Dataset/PAMAP2_Dataset/' 
data_path = os.path.join(data_dir, 'pamap_small.csv')

In [3]:
df_init = pd.read_csv(data_path)
df_init.shape

(248827, 41)

In [4]:
df_init = pd.read_csv(data_path)
print(f'full df shape {df_init.shape}')
df_train, df_test = train_test_split(df_init, test_size=0.3, random_state=42) # do not change
del df_init
print(f'{df_train.shape = }, {df_test.shape = }')

full df shape (248827, 41)
df_train.shape = (174178, 41), df_test.shape = (74649, 41)


In [5]:
# add some real preprocessing logic instead of that
class PreprocessingPipeline:
    def __init__(self, prep_param=None):
        self.prep_param = prep_param
    
    def fit(self, X):
        pass
    
    def transform(self, X):
        return X.fillna(0)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    

In [6]:
prep_pipeline = PreprocessingPipeline()
X_train = prep_pipeline.fit_transform(df_train.drop(columns='activityID'))
y_train = df_train['activityID']

In [7]:
from sklearn.cluster import KMeans


param_grid = {
    'n_clusters': [2, 3],  # тут нужно осмысленно выборать параметры
    'n_init': [5, 10]
}
grid_search = GridSearchCV(KMeans(), param_grid, cv=5, scoring='adjusted_rand_score')

In [8]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KMeans(),
             param_grid={'n_clusters': [2, 3], 'n_init': [5, 10]},
             scoring='adjusted_rand_score')

In [9]:
grid_search.best_score_

0.09352395556853393

In [10]:
grid_search.best_estimator_

KMeans(n_clusters=3, n_init=10)

In [14]:
full_model = KMeans(**grid_search.best_params_).fit(X_train, y_train)

In [15]:
X_test = prep_pipeline.transform(df_test.drop(columns='activityID'))
y_test = df_test['activityID']

In [16]:
preds = full_model.predict(X_test)

In [24]:
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score

ari = adjusted_rand_score(y_test, preds)
armi = adjusted_mutual_info_score(y_test, preds)

print(f'{ari = :.4f}, {armi = :.4f}')

ari = 0.0939, armi = 0.2289
